# Wine Sommelier Agent

LangGraph workflow with:
- Custom state (conversation summary, final answer, intermediate data)
- 8 nodes with 3 conditional edges
- RAG as a tool
- Evaluator-Optimizer pattern
- Langfuse tracing

## 1. Setup and Imports

In [10]:
import json
from typing import Annotated, Optional, List, Literal
from dotenv import load_dotenv
import os

from langchain_openai import AzureOpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, AnyMessage
from langchain_core.tools import tool

from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END, add_messages
from langgraph.prebuilt import ToolNode, create_react_agent

from langfuse import Langfuse
from langfuse.langchain import CallbackHandler

load_dotenv()
print("Imports loaded successfully")

Imports loaded successfully


## 2. Initialize Models and RAG

In [11]:
# Initialize embeddings and LLM
AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT = os.getenv("EMBEDDING_DEPLOYMENT_NAME")
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY missing"

embeddings = AzureOpenAIEmbeddings(
    deployment=AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT,
    base_url=None,
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

llm = ChatOpenAI(model="gpt-4.1", temperature=0.0)
print("LLM ready")

LLM ready


In [12]:
# Load wine vector store
PERSIST_DIR = "./faiss_index_wine"

# Check if wine index exists, otherwise use the movie index
if os.path.exists(PERSIST_DIR):
    vector_store = FAISS.load_local(PERSIST_DIR, embeddings, allow_dangerous_deserialization=True)
    print(f"Loaded wine vector store: {len(vector_store.index_to_docstore_id)} documents")
else:
    # Fallback to movie index if wine not available
    PERSIST_DIR = "./faiss_index"
    vector_store = FAISS.load_local(PERSIST_DIR, embeddings, allow_dangerous_deserialization=True)
    print(f"Loaded fallback vector store: {len(vector_store.index_to_docstore_id)} documents")

Loaded wine vector store: 5709 documents


In [13]:
# Create RAG chain
def format_docs(docs):
    return "\n\n".join(f"{json.dumps(d.metadata)}: {d.page_content}" for d in docs)

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a knowledgeable wine sommelier. Use the provided wine database context to answer questions about wines, provide recommendations, and suggest food pairings. If you don't find relevant information, say so."),
    ("human", "Question: {question}\n\nWine Database Context:\n{context}\n\nAnswer:")
])

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("RAG chain constructed")

RAG chain constructed


## 3. Langfuse Setup

In [14]:
# Langfuse tracing (optional - will work without if not configured)
try:
    langfuse = Langfuse(
        secret_key=os.getenv("LANGFUSE_SECRET_KEY", "sk-lf-ad603960-aad1-482e-8c78-79eda9b5e8f3"),
        public_key=os.getenv("LANGFUSE_PUBLIC_KEY", "pk-lf-c209d32a-cea6-473f-8ce4-f8c7d6a4bd70"),
        host=os.getenv("LANGFUSE_HOST", "http://127.0.0.1:3000")
    )
    langfuse_handler = CallbackHandler()
    print("Langfuse configured")
except Exception as e:
    langfuse_handler = None
    print(f"Langfuse not configured: {e}")

Langfuse configured


Exception while exporting Span.
Traceback (most recent call last):
  File "C:\Users\cheme\OneDrive\Документы\AI\КУРСИ та МАТЕРІАЛИ\Курс Langchain+phyton\SSU LLM Curs\sswu-llm-engineering\venv\lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
  File "C:\Users\cheme\OneDrive\Документы\AI\КУРСИ та МАТЕРІАЛИ\Курс Langchain+phyton\SSU LLM Curs\sswu-llm-engineering\venv\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "C:\Users\cheme\OneDrive\Документы\AI\КУРСИ та МАТЕРІАЛИ\Курс Langchain+phyton\SSU LLM Curs\sswu-llm-engineering\venv\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\cheme\OneDrive\Документы

## 4. Define Tools

In [15]:
@tool(description="Search wine database for recommendations, information about wines, or pairing suggestions. Use this for any wine-related queries.")
def wine_search(query: Annotated[str, "Search query about wines, preferences, regions, or pairings"]) -> str:
    """Uses RAG chain to find relevant wines from the database"""
    print(f"Wine search called: {query}")
    result = rag_chain.invoke(query)
    return result


@tool(description="Get weather information for wine serving temperature recommendations")
def get_weather(
    city: Annotated[str, "City name for weather lookup"]
) -> str:
    """Returns weather info to help with serving temperature suggestions"""
    import random
    temp = random.randint(15, 35)
    conditions = random.choice(["Sunny", "Cloudy", "Warm", "Cool"])
    
    suggestion = ""
    if temp > 25:
        suggestion = "Serve white wines well-chilled (7-10°C) and reds slightly cool (14-16°C)"
    else:
        suggestion = "Standard serving temperatures work well today"
    
    return f"Weather in {city}: {temp}°C, {conditions}. {suggestion}"


@tool(description="Calculate wine and food pairing compatibility score")
def pairing_score(
    wine_type: Annotated[str, "Type of wine: red, white, rosé, sparkling, or dessert"],
    food: Annotated[str, "Food dish to pair with the wine"]
) -> str:
    """Returns a compatibility score for wine-food pairing"""
    # Pairing rules
    pairings = {
        ("red", "steak"): (95, "Excellent! Tannins complement the rich meat"),
        ("red", "beef"): (95, "Excellent! Classic pairing"),
        ("red", "lamb"): (90, "Great match with the gamey flavors"),
        ("red", "cheese"): (85, "Good pairing, especially aged cheeses"),
        ("red", "pasta"): (80, "Works well with tomato-based sauces"),
        ("white", "fish"): (92, "Perfect! Acidity complements seafood"),
        ("white", "seafood"): (92, "Excellent pairing"),
        ("white", "chicken"): (85, "Good match for lighter preparations"),
        ("white", "salad"): (80, "Refreshing combination"),
        ("sparkling", "appetizers"): (90, "Great for starting a meal"),
        ("sparkling", "oysters"): (95, "Classic luxurious pairing"),
        ("rosé", "salad"): (85, "Light and refreshing"),
        ("dessert", "chocolate"): (88, "Sweet wines complement desserts"),
    }
    
    key = (wine_type.lower(), food.lower())
    if key in pairings:
        score, reason = pairings[key]
    else:
        score = 70
        reason = "Moderate pairing - can work but not ideal"
    
    return f"Pairing Score: {score}/100\nWine: {wine_type}\nFood: {food}\nAnalysis: {reason}"


@tool(description="Save a wine recommendation to user's favorites list")
def save_favorite(
    wine_name: Annotated[str, "Name of the wine to save"],
    notes: Annotated[str, "User's notes about why they liked this wine"]
) -> str:
    """Saves wine to favorites for future reference"""
    print(f"Saving to favorites: {wine_name}")
    return f"Successfully saved '{wine_name}' to your favorites!\nNotes: {notes}"


print("Tools defined: wine_search, get_weather, pairing_score, save_favorite")

Tools defined: wine_search, get_weather, pairing_score, save_favorite


## 5. Define Custom State

In [17]:
class WineSommelierState(BaseModel):
    """Custom state for Wine Sommelier workflow"""
    
    # Core message history
    messages: Annotated[List[AnyMessage], add_messages] = Field(default_factory=list)
    
    # Conversation management
    conversation_summary: Optional[str] = None
    
    # Final output
    final_answer: Optional[str] = None
    
    # Intermediate data
    user_intent: Optional[str] = None
    wine_context: Optional[str] = None
    recommendation_draft: Optional[str] = None
    quality_score: Optional[float] = None
    
    # Control flow
    remaining_steps: int = 10
    needs_improvement: bool = False
    optimization_attempts: int = 0

print("WineSommelierState defined")

WineSommelierState defined


## 6. Implement Nodes

In [18]:
# Node 1: Intent Classifier
def intent_classifier(state: WineSommelierState) -> dict:
    """Classifies user intent to route appropriately"""
    
    last_message = ""
    for msg in reversed(state.messages):
        if isinstance(msg, HumanMessage):
            last_message = msg.content
            break
    
    classification_prompt = f"""Classify the user's intent into ONE of these categories:
    - RECOMMENDATION: User wants wine recommendations
    - PAIRING: User wants food-wine pairing advice  
    - INFORMATION: User wants to learn about wines, regions, or varieties
    - GENERAL: General conversation or greeting
    
    User message: {last_message}
    
    Return ONLY the category name, nothing else."""
    
    response = llm.invoke([SystemMessage(classification_prompt)])
    intent = response.content.strip().upper()
    
    # Validate intent
    valid_intents = ["RECOMMENDATION", "PAIRING", "INFORMATION", "GENERAL"]
    if intent not in valid_intents:
        intent = "GENERAL"
    
    print(f"[Intent Classifier] Classified as: {intent}")
    return {"user_intent": intent}


# Node 2: RAG Retriever
def rag_retriever(state: WineSommelierState) -> dict:
    """Retrieves relevant wine context using RAG"""
    
    query = ""
    for msg in reversed(state.messages):
        if isinstance(msg, HumanMessage):
            query = msg.content
            break
    
    # Get relevant documents
    docs = retriever.invoke(query)
    context = format_docs(docs)
    
    print(f"[RAG Retriever] Retrieved {len(docs)} documents, {len(context)} chars")
    return {"wine_context": context}


# Node 3: Evaluator (Evaluator-Optimizer Pattern)
def evaluator(state: WineSommelierState) -> dict:
    """Evaluates quality of the recommendation"""
    
    # Get the last AI message as the draft
    draft = None
    for msg in reversed(state.messages):
        if isinstance(msg, AIMessage) and msg.content:
            draft = msg.content
            break
    
    if not draft:
        print("[Evaluator] No draft to evaluate")
        return {
            "quality_score": 0.0,
            "needs_improvement": True,
            "recommendation_draft": ""
        }
    
    eval_prompt = f"""Evaluate this wine recommendation on a scale of 0-100.

Recommendation:
{draft}

Scoring criteria:
- Relevance to user query (0-25 points)
- Specificity of wine details (grape, region, vintage) (0-25 points)
- Helpfulness of serving/pairing suggestions (0-25 points)
- Clarity and engaging presentation (0-25 points)

Return ONLY a single number (the total score), nothing else."""
    
    response = llm.invoke([SystemMessage(eval_prompt)])
    
    try:
        score = float(response.content.strip())
        score = max(0, min(100, score))  # Clamp to 0-100
    except:
        score = 70.0
    
    needs_improvement = score < 75 and state.optimization_attempts < 2
    
    print(f"[Evaluator] Score: {score}/100, needs improvement: {needs_improvement}")
    return {
        "quality_score": score,
        "needs_improvement": needs_improvement,
        "recommendation_draft": draft
    }


# Node 4: Optimizer
def optimizer(state: WineSommelierState) -> dict:
    """Improves recommendation based on evaluator feedback"""
    
    optimize_prompt = f"""Improve this wine recommendation to score higher.

Current recommendation (scored {state.quality_score}/100):
{state.recommendation_draft}

Improvements needed:
- Add more specific wine details (grape varieties, regions, producers)
- Include serving temperature and glass recommendations
- Add food pairing suggestions
- Make it more engaging and personalized

Provide the improved recommendation:"""
    
    response = llm.invoke([
        SystemMessage("You are a master sommelier improving wine recommendations to be more helpful and specific."),
        HumanMessage(optimize_prompt)
    ])
    
    improved = response.content
    
    print(f"[Optimizer] Improved recommendation (attempt {state.optimization_attempts + 1})")
    return {
        "recommendation_draft": improved,
        "optimization_attempts": state.optimization_attempts + 1,
        "messages": [AIMessage(content=improved)]
    }


# Node 5: Answer Saver
def answer_saver(state: WineSommelierState) -> dict:
    """Saves final answer to state"""
    
    final = state.recommendation_draft
    if not final:
        for msg in reversed(state.messages):
            if isinstance(msg, AIMessage) and msg.content:
                final = msg.content
                break
    
    print(f"[Answer Saver] Final answer saved ({len(final) if final else 0} chars)")
    return {"final_answer": final}


# Node 6: Summarizer
def summarizer(state: WineSommelierState) -> dict:
    """Creates conversation summary for context management"""
    
    if len(state.messages) < 3:
        return {}
    
    # Get last few messages for summary
    recent_messages = state.messages[-6:] if len(state.messages) > 6 else state.messages
    
    summary_prompt = """Summarize this wine consultation conversation in 2-3 sentences:
    - What the user was looking for
    - What recommendations were made
    - Any specific preferences mentioned"""
    
    response = llm.invoke([
        SystemMessage(summary_prompt),
        *recent_messages
    ])
    
    summary = response.content
    print(f"[Summarizer] Created summary: {summary[:100]}...")
    return {"conversation_summary": summary}


print("Nodes defined: intent_classifier, rag_retriever, evaluator, optimizer, answer_saver, summarizer")

Nodes defined: intent_classifier, rag_retriever, evaluator, optimizer, answer_saver, summarizer


## 7. Define Conditional Edge Functions

In [19]:
def route_by_intent(state: WineSommelierState) -> Literal["rag_retriever", "agent"]:
    """Routes based on classified intent"""
    
    if state.user_intent in ["RECOMMENDATION", "PAIRING", "INFORMATION"]:
        print(f"[Router] Intent '{state.user_intent}' -> RAG Retriever")
        return "rag_retriever"
    
    print(f"[Router] Intent '{state.user_intent}' -> Agent directly")
    return "agent"


def should_continue_agent(state: WineSommelierState) -> Literal["tools", "evaluator"]:
    """Determines if agent should call tools or proceed to evaluation"""
    
    last_message = state.messages[-1] if state.messages else None
    
    if last_message and hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        print(f"[Router] Agent has tool calls -> Tools")
        return "tools"
    
    print(f"[Router] Agent finished -> Evaluator")
    return "evaluator"


def route_after_evaluation(state: WineSommelierState) -> Literal["optimizer", "answer_saver"]:
    """Routes based on evaluation score"""
    
    if state.needs_improvement:
        print(f"[Router] Score {state.quality_score} < 75 -> Optimizer")
        return "optimizer"
    
    print(f"[Router] Score {state.quality_score} >= 75 -> Answer Saver")
    return "answer_saver"


print("Conditional edge functions defined")

Conditional edge functions defined


## 8. Build the Graph

In [20]:
def build_wine_sommelier_graph():
    """Builds the complete Wine Sommelier workflow"""
    
    # Define tools list
    tools = [wine_search, get_weather, pairing_score, save_favorite]
    tool_node = ToolNode(tools)
    
    # Create the agent
    agent = create_react_agent(
        model=llm,
        tools=tools
    )
    
    # Build workflow
    workflow = StateGraph(WineSommelierState)
    
    # Add all nodes (8 nodes total)
    workflow.add_node("intent_classifier", intent_classifier)
    workflow.add_node("rag_retriever", rag_retriever)
    workflow.add_node("agent", agent)
    workflow.add_node("tools", tool_node)
    workflow.add_node("evaluator", evaluator)
    workflow.add_node("optimizer", optimizer)
    workflow.add_node("answer_saver", answer_saver)
    workflow.add_node("summarizer", summarizer)
    
    # Add edges
    workflow.add_edge(START, "intent_classifier")
    
    # Conditional edge 1: Route by intent
    workflow.add_conditional_edges(
        "intent_classifier",
        route_by_intent,
        {
            "rag_retriever": "rag_retriever",
            "agent": "agent"
        }
    )
    
    workflow.add_edge("rag_retriever", "agent")
    
    # Conditional edge 2: Agent decides tools or evaluate
    workflow.add_conditional_edges(
        "agent",
        should_continue_agent,
        {
            "tools": "tools",
            "evaluator": "evaluator"
        }
    )
    
    workflow.add_edge("tools", "agent")
    
    # Conditional edge 3: Evaluate and optimize loop
    workflow.add_conditional_edges(
        "evaluator",
        route_after_evaluation,
        {
            "optimizer": "optimizer",
            "answer_saver": "answer_saver"
        }
    )
    
    workflow.add_edge("optimizer", "agent")
    workflow.add_edge("answer_saver", "summarizer")
    workflow.add_edge("summarizer", END)
    
    # Compile with optional langfuse tracing
    if langfuse_handler:
        return workflow.compile().with_config(callbacks=[langfuse_handler])
    return workflow.compile()


# Build the graph
graph = build_wine_sommelier_graph()
print("Wine Sommelier graph built successfully!")
print("Nodes: 8, Conditional edges: 3")

Wine Sommelier graph built successfully!
Nodes: 8, Conditional edges: 3


## 9. Test the Agent

In [21]:
# Test 1: Wine recommendation
print("="*60)
print("TEST 1: Wine Recommendation Request")
print("="*60)

result = graph.invoke({
    "messages": [
        SystemMessage("You are a knowledgeable and friendly wine sommelier assistant. Help users discover wines they'll love."),
        HumanMessage("I'm looking for a red wine to pair with grilled steak for a dinner party. Budget around $30-50.")
    ]
})

final_state = WineSommelierState(**result)
print("\n" + "="*60)
print("RESULTS:")
print("="*60)
print(f"Intent: {final_state.user_intent}")
print(f"Quality Score: {final_state.quality_score}")
print(f"Optimization Attempts: {final_state.optimization_attempts}")
print(f"\nFinal Answer:\n{final_state.final_answer}")
print(f"\nConversation Summary:\n{final_state.conversation_summary}")

TEST 1: Wine Recommendation Request
[Intent Classifier] Classified as: PAIRING
[Router] Intent 'PAIRING' -> RAG Retriever
[RAG Retriever] Retrieved 5 documents, 1585 chars
Wine search called: red wine for pairing with grilled steak, budget $30-50
[Router] Agent finished -> Evaluator
[Evaluator] Score: 93.0/100, needs improvement: False
[Router] Score 93.0 >= 75 -> Answer Saver
[Answer Saver] Final answer saved (570 chars)
[Summarizer] Created summary: The user was looking for a red wine in the $30–$50 range to pair with grilled steak at a dinner part...

RESULTS:
Intent: PAIRING
Quality Score: 93.0
Optimization Attempts: 0

Final Answer:
A great red wine for grilled steak within your $30–$50 budget is the **Venta del Puerto 2006 N° 12 Selección Especial Red (Valencia, Spain)**. This Spanish blend is robust, balanced, and earns high ratings—making it an excellent match for the rich flavors and char of steak. It should be available around $35.

If you prefer Italian wines, consider a Bru

In [22]:
# Test 2: Food pairing query
print("="*60)
print("TEST 2: Food Pairing Query")
print("="*60)

result = graph.invoke({
    "messages": [
        SystemMessage("You are a knowledgeable and friendly wine sommelier assistant."),
        HumanMessage("What wine would go well with salmon? I prefer lighter wines.")
    ]
})

final_state = WineSommelierState(**result)
print(f"\nIntent: {final_state.user_intent}")
print(f"Quality Score: {final_state.quality_score}")
print(f"\nFinal Answer:\n{final_state.final_answer[:500]}..." if len(final_state.final_answer or "") > 500 else f"\nFinal Answer:\n{final_state.final_answer}")

TEST 2: Food Pairing Query
[Intent Classifier] Classified as: PAIRING
[Router] Intent 'PAIRING' -> RAG Retriever
[RAG Retriever] Retrieved 5 documents, 2690 chars
[Router] Agent finished -> Evaluator
[Evaluator] Score: 92.0/100, needs improvement: False
[Router] Score 92.0 >= 75 -> Answer Saver
[Answer Saver] Final answer saved (1513 chars)
[Summarizer] Created summary: The user was looking for a wine pairing for salmon and mentioned a preference for lighter wines. I r...

Intent: PAIRING
Quality Score: 92.0

Final Answer:
Salmon is a versatile fish that pairs well with a variety of wines, but lighter wines complement it particularly well. Here are some excellent choices for pairing with salmon, especially if you prefer lighter wines:

### 1. **Pinot Noir**
- **Why:** This is a classic pairing! Although it’s a red, Pinot Noir is light-bodied with bright acidity and red fruit flavors that work beautifully with the richness of salmon.
- **Best Style:** Opt for a cool-climate Pinot Noir (

In [23]:
# Test 3: General wine information
print("="*60)
print("TEST 3: Wine Information Query")
print("="*60)

result = graph.invoke({
    "messages": [
        SystemMessage("You are a knowledgeable wine sommelier assistant."),
        HumanMessage("Tell me about Pinot Noir wines from Oregon.")
    ]
})

final_state = WineSommelierState(**result)
print(f"\nIntent: {final_state.user_intent}")
print(f"Quality Score: {final_state.quality_score}")
print(f"\nFinal Answer:\n{final_state.final_answer[:500]}..." if len(final_state.final_answer or "") > 500 else f"\nFinal Answer:\n{final_state.final_answer}")

TEST 3: Wine Information Query
[Intent Classifier] Classified as: INFORMATION
[Router] Intent 'INFORMATION' -> RAG Retriever
[RAG Retriever] Retrieved 5 documents, 2477 chars
[Router] Agent finished -> Evaluator
[Evaluator] Score: 92.0/100, needs improvement: False
[Router] Score 92.0 >= 75 -> Answer Saver
[Answer Saver] Final answer saved (2202 chars)
[Summarizer] Created summary: **Summary:**  
- The user was seeking information about Pinot Noir wines from Oregon.
- The assistan...

Intent: INFORMATION
Quality Score: 92.0

Final Answer:
Pinot Noir from Oregon is renowned for its elegance, complexity, and distinctive sense of place, especially from the Willamette Valley, the state’s most famous wine region for this grape. Here’s a detailed look:

## **Overview**
- **Grape Variety:** Pinot Noir is a thin-skinned, red wine grape variety originally from Burgundy, France.
- **Region:** Oregon, USA — Willamette Valley is the star region, but other areas like Umpqua Valley and Rogue Valley 

## 10. Interactive Chat Function

In [24]:
def chat_with_sommelier(user_message: str) -> str:
    """Simple function to chat with the Wine Sommelier"""
    
    result = graph.invoke({
        "messages": [
            SystemMessage("You are a knowledgeable and friendly wine sommelier. Help users discover wines they'll love."),
            HumanMessage(user_message)
        ]
    })
    
    final_state = WineSommelierState(**result)
    return final_state.final_answer or "I couldn't generate a response. Please try again."


# Example usage
response = chat_with_sommelier("Recommend a wine for a romantic dinner")
print(response)

[Intent Classifier] Classified as: RECOMMENDATION
[Router] Intent 'RECOMMENDATION' -> RAG Retriever
[RAG Retriever] Retrieved 5 documents, 1105 chars
[Router] Agent finished -> Evaluator
[Evaluator] Score: 80.0/100, needs improvement: False
[Router] Score 80.0 >= 75 -> Answer Saver
[Answer Saver] Final answer saved (1178 chars)
[Summarizer] Created summary: The user was looking for a wine recommendation suitable for a romantic dinner. Recommendations inclu...
A romantic dinner calls for a wine that sets the mood—something elegant, delicious, and a little special. The best choice depends on your meal and personal taste, but here are a few romantic classics that never fail:

**Red Wine:**  
- **Pinot Noir** – Silky, smooth, and aromatic. Its light tannins and notes of cherry, rose, and spice pair beautifully with dishes like roasted chicken, duck, or mushroom risotto.
- **Chianti Classico** – Vibrant and food-friendly, perfect with Italian cuisine or tomato-based dishes.

**Sparkling Win

## Summary

### Implemented Features:
- **Custom State**: 11 fields including conversation_summary, final_answer, intermediate data
- **8 Nodes**: intent_classifier, rag_retriever, agent, tools, evaluator, optimizer, answer_saver, summarizer
- **3 Conditional Edges**: by intent, after agent, after evaluation
- **RAG as Tool**: wine_search uses the RAG chain
- **Evaluator-Optimizer Pattern**: Quality feedback loop from Anthropic's guide
- **Langfuse Tracing**: Optional observability
- **Conversation Summarization**: For token optimization